# Four Nations

What if our fantasy league happened over the four nations? How would everyone do, and what 'awards' would everyone get?

In [ ]:
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import pandas as pd

In [ ]:
import sys
sys.path.append("..")
from base_python_files.api_info import *
from base_python_files.credentials import *
from base_python_files.api_functions import *


In [ ]:
gameIDs = []
month = '02'
days = range(10,22)
for day in tqdm(days):
    if len(str(day))==1:
        day = "0"+str(day)
    day_url = f"https://api-web.nhle.com/v1/score/2025-{month}-{day}"
    r=requests.get(day_url)
    try:
        gameIDs+=[game["id"] for game in r.json()['games']]
    except:
        print(month,day)
        pass

In [ ]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

In [ ]:
def convert_to_default(name):
    parts = name.split()
    return f"{parts[0][0]}. {' '.join(parts[1:])}"


In [ ]:
players = data[~data['LeagueTeamID'].isna()]
players['DefaultName'] = players['Name'].apply(convert_to_default)
players.index=players['DefaultName']

In [ ]:
players_ids = players['LeagueTeamID'].to_dict()

In [ ]:
score_cats = ['starter','saves','shotsAgainst','goals','assists','toi','players','sog','hits','blockedShots','pim','plusMinus','faceoffWinningPctg','faceoffplayers','wins','powerPlayGoals','powerPlayAssists']

In [ ]:
score_dict = {
    cat:0 for cat in score_cats
}

In [ ]:
scores = {
    k:score_dict.copy() for k in range(1,9)
}

In [ ]:
all_cats = []
all_tois = {}
for game in tqdm(gameIDs):
    game_url = f"https://api-web.nhle.com/v1/gamecenter/{game}/boxscore"
    try:
        all_player_stats=requests.get(game_url).json()['playerByGameStats']
        for team_type in ['awayTeam','homeTeam']:
            for position in ['forwards','defense','goalies']:
                for player in all_player_stats[team_type][position]:
                    if player['name']['default'] in list(players_ids.keys()):
                        team_id = int(players_ids[player['name']['default']])
                        for cat in player:
                            if cat not in all_cats:
                                all_cats.append(cat)
                            if cat in score_cats and cat!='toi':
                                scores[team_id][cat]+=player[cat]
                            elif cat=='toi':
                                temp_name = player['name']['default']
                                player_time = 60*int(player[cat][0:2])+int(player[cat][3:])
                                scores[team_id][cat]+=player_time
                                if temp_name in all_tois:
                                    all_tois[temp_name].append(player_time)
                                    
                                else:
                                    all_tois[temp_name] = [player_time]
                            if cat=='faceoffWinningPctg' and player[cat]>0:
                                scores[team_id]['faceoffplayers']+=1
                            if cat=='decision':
                                if player[cat]=='W':
                                    scores[team_id]['wins']+=1
                        scores[team_id]['players']+=1
                    else:
                        print('PLAYER NOT IN A TEAM:', player['name']['default'])      
    except KeyError:
        print('No game', game)

In [ ]:
all_tois_average = {}
for player in all_tois:
    all_tois_average[player]=np.mean(all_tois[player])

In [ ]:
all_tois_average_list = list(all_tois_average.items())
all_tois_average_list.sort(key=lambda x:x[1])

In [ ]:
all_tois_average_list.reverse()

In [ ]:
all_tois_average_list

In [ ]:
for team_id in range(1,9):
    scores[team_id]['atoi']=scores[team_id]['toi']/scores[team_id]['players']/60
    if scores[team_id]['shotsAgainst']==0:
        scores[team_id]['savePctg']=0
    else:
        scores[team_id]['savePctg']=scores[team_id]['saves']/scores[team_id]['shotsAgainst']
    scores[team_id]['powerPlayPoints']=scores[team_id]['powerPlayGoals']+scores[team_id]['powerPlayAssists']

In [ ]:
final_score_cats = ['powerPlayPoints','starter','savePctg','goals','assists','atoi','sog','hits','blockedShots','pim','plusMinus','faceoffWinningPctg','wins']

In [ ]:

for cat in scores[1]:
    highest = 0
    highest_team = []
    for i in range(1,9):
        if scores[i][cat]/scores[i]['players']>highest:
            highest = scores[i][cat]/scores[i]['players']
            highest_team=[i]
        elif scores[i][cat]/scores[i]['players']==highest:
            highest_team.append(i)

    print(cat,highest_team,highest)

In [ ]:

for cat in scores[1]:
    highest = 0
    highest_team = []
    for i in range(1,9):
        if scores[i][cat]>highest:
            highest = scores[i][cat]
            highest_team=[i]
        elif scores[i][cat]==highest:
            highest_team.append(i)

    print(cat,highest_team,highest)

In [ ]:
for team_id in range(1,9):
    scores[team_id] = {k:scores[team_id][k] for k in final_score_cats}

In [ ]:
team_week_scores = {}
team_matrix = np.zeros((8,8))
min_weeks = 0
weeks = 18

for team_id in scores:
    wins = 0
    draws = 0
    losses = 0
    total_cat_wins = 0
    total_cat_losses = 0
    total_cat_draws = 0
    for second_team_id in scores:
        if team_id==second_team_id:
            continue
        cats_won = 0
        cats_lost = 0
        team = scores[team_id]
        second_team = scores[second_team_id]
        for score_cat in team:
            if team[score_cat]>second_team[score_cat]:
                cats_won+=1
            elif team[score_cat]<second_team[score_cat]:
                cats_lost+=1
        total_cat_wins+=cats_won
        total_cat_losses+=cats_lost
        total_cat_draws+=13-cats_won-cats_lost
        if cats_won>cats_lost:
            wins+=1
            team_matrix[team_id-1,second_team_id-1]+=1
            team_matrix[second_team_id-1,team_id-1]-=1
        elif cats_won<cats_lost:
            losses+=1
            team_matrix[team_id-1,second_team_id-1]-=1
            team_matrix[second_team_id-1,team_id-1]+=1
        else:
            draws+=1
        # if week == 15 and team_id==8:
            # print('ID', second_team_id,'Winning',cats_won,'Losing',cats_lost)
    if team_id in team_week_scores:
        team_week_scores[team_id]+=np.array([wins,losses,draws,total_cat_wins,total_cat_losses,total_cat_draws])
    else:
        team_week_scores[team_id]=np.array([wins,losses,draws,total_cat_wins,total_cat_losses,total_cat_draws])
    

In [ ]:
sorted(team_week_scores.items(),key=lambda x:x[1][0],reverse=True)

In [ ]:
plt.imshow(team_matrix)
plt.colorbar()